<a href="https://colab.research.google.com/github/kvolle/SpeedChallenge/blob/single_mem_test/Tiny_Kyle's_Speed_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Comma AI Speed Challenge**

  This notebook will contain (hopefully) all of the functions you need to import the data into your model.

  ***Be sure to train with GPU acceleration enabled***

**Import Statements**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
x = np.load("./drive/My Drive/commai_dataset/data/data_001.npy")

**Custom Data Generator**

This works (I think) for giving two sequential images to a Keras Functional model as well as the velocity associated with the second image.

At this point, don't worry about how this works. If you need something changed or fixed, just ask. This is the boring part anyways.

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.direct = "./drive/My Drive/commai_dataset/"

    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.list_IDs) #int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        
        # Find list of IDs
        list_IDs_temp = self.list_IDs[index]
        # Generate data
        #print(index)
        #print(list_IDs_temp)
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.load(self.direct+"data/data_" + list_IDs_temp +".npy")
        #x1 = X[0:101,:,:,:]/255.
        #x1 = x1 - np.mean(x1, axis=0)
        x2 = X[1:102,:,:,:]/255.
        x2 = x2 - np.mean(x2, axis=0)
        y = np.load(self.direct+"labels/label_" + list_IDs_temp +".npy")
        y = y[1:102]#/10.

        #return [x1, x2], y
        return x2, y

**Define custom loss function**

This is not well tested, neither is it optimized. You might not even want to use this function.

Keras backend functions are a powerful tool for writing custom loss functions. To define a loss function it just has to accept *y_true* and *y_pred* as arguments and return a float.

To use your new loss function, change the argument in *model.compile()*.

In [ ]:
def mean_sq_err(y_true, y_pred):
  #y_pred = 15. + y_pred*15.
  return tf.keras.backend.mean(tf.keras.backend.square(y_true - y_pred)) #+ metric_var(y_true, y_pred)#1./(0.001+tf.keras.backend.var(y_pred))

In [ ]:
def mean_psuedo_huber(y_true, y_pred):
  scale = 10
  huber = scale*scale*(tf.keras.backend.sqrt(1+tf.keras.backend.square((y_true-y_pred)/scale)) - 1)
  return tf.keras.backend.mean(huber)

In [ ]:
def metric_var(y_true, y_pred):
  #return 1./(0.001+tf.keras.backend.var(y_pred))
  epsilon = 0.001
  return tf.keras.backend.maximum((epsilon + tf.keras.backend.var(y_true))/(epsilon+tf.keras.backend.var(y_pred)), 1)

In [ ]:
def mean_psuedo_huber_var(y_true, y_pred):
  scale_factor = 10.
  return scale_factor*mean_psuedo_huber(y_true, y_pred) + metric_var(y_true, y_pred)

**Define the test-train split and create the Data Generator**

In [ ]:
params = {'dim': (110,320),
          'batch_size': 101,
          'n_classes': 1,
          'n_channels': 3,
          'shuffle': False}

train_data = []
train_label = []
valid_data = []
valid_label = []

for i in range(70):
    train_data.append("%03d" %i)
    train_data.append("%03d" %(i+100))

for i in range(70, 100):
    valid_data.append("%03d" %i)
    valid_data.append("%03d" %(i+100))

partition={'train':train_data, 'validation':valid_data}
labels = {'train': train_label,'validation':valid_label}

training_generator = DataGenerator(partition['train'], labels['train'], **params)
validation_generator =DataGenerator(partition['validation'], labels['validation'], **params)

**Define the input layers**

In [ ]:
input_A = tf.keras.layers.Input(shape=(110, 320, 3), name="first_image")
#input_B = tf.keras.layers.Input(shape=(110, 320, 3), name="second_image")

**Define the model**

This model is a particularly ridiculous placeholder. Make yours better

In [ ]:
def create_side():
  side_input = tf.keras.layers.Input(shape=(110, 320, 3), name="side_input")
  pre_split_conv_1 = tf.keras.layers.Conv2D(16, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l1_l2())(side_input)
  pre_split_conv_1 = tf.keras.layers.MaxPool2D((2,2))(pre_split_conv_1)
  side_output = tf.keras.layers.Conv2D(32, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l1_l2())(pre_split_conv_1)
  return tf.keras.models.Model(inputs = side_input, outputs = side_output)

In [ ]:
side = create_side() 
res_a = side(input_A)
#res_b = side(input_B)
#concat = tf.keras.layers.concatenate([res_a, res_b])
conv_1 = tf.keras.layers.Conv2D(164, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l1_l2())(res_a)
conv_1 = tf.keras.layers.MaxPool2D((3,3))(conv_1)
conv_2 = tf.keras.layers.Conv2D(32, (3,3), padding='same', kernel_regularizer=tf.keras.regularizers.l1_l2())(conv_1)
conv_2 = tf.keras.layers.MaxPool2D((3,23))(conv_2)
flat = tf.keras.layers.Flatten()(conv_2)
dense_1 = tf.keras.layers.Dense(1024)(flat)
#dense_2 = tf.keras.layers.Dense(64)(dense_1)
dense_3 = tf.keras.layers.Dense(1, activation='relu')(dense_1)
model = tf.keras.models.Model(inputs=[input_A, input_B], outputs=dense_3)

**How ridiculous? Run this next cell and find out.**

(Note how many parameters there are, at least it trains fast)

In [ ]:
model.summary()

**Declare the optimizer and loss function, then compile your *less ridiculous*  model**

In [ ]:
_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=_optimizer, loss = mean_psuedo_huber,  metrics=[metric_var, mean_sq_err])

**Train using the fit_generator**

In [ ]:
%load_ext tensorboard

In [ ]:
logdir = "./drive/My Drive/SpeedChallenge/Logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1, update_freq=1250)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="./drive/My Drive/SpeedChallenge/Checkpoints/removed_activations_changed_pool{epoch}.h5", save_weights_only=True,  verbose=0)

In [ ]:
model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=12, initial_epoch=0,callbacks=[tensorboard_callback, cp_callback])

In [ ]:
%tensorboard --logdir ./drive/My\ Drive/SpeedChallenge/Logs/20200630-032031/

In [ ]:
x = np.load("./drive/My Drive/commai_dataset/data/data_049.npy")
truth = np.load("./drive/My Drive/commai_dataset/labels/label_049.npy")
x1 = x[0:20,:,:,:]/255.
x2 = x[1:21,:,:,:]/255.
y = model.predict([x1, x2])

In [ ]:
print(y)
print(truth[1:21])